In [ ]:
# TODO: Load the database

from google.colab import files
import pandas as pd

# Opens a file picker to select SQLite file
uploaded = files.upload()
# After loading, it returns the dictionary {"file_name ": contents in bytes}
# ===============================

for filename in uploaded.keys(): #list of all names of uploaded files
    print("Uploaded file:", "database")


In [ ]:
import sqlite3
# Importing a module for working with SQLite databases

from sqlalchemy import create_engine, text
# creates an “engine", an object that manages the connection to the database to:

# connect to the database (SQLite, PostgreSQL, MySQL, etc.)
# send SQL queries
# get results

# text() is used to write a “raw” SQL query (plain SQL)

DB_PATH = "database.sqlite" #a string with the path to the SQLite database file.
# The DB_PATH variable is needed to conveniently refer to this file in the code later
# === 2) Create an SQLite DB via SQLAlchemy and export the DataFrame ===
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)

conn = sqlite3.connect(DB_PATH)
# The sqlite3 module allows to work with SQLite databases directly from Python.
# The connect() function opens a connection to the specified database file.

# List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in database:")
print(tables)


In [ ]:
query = """
SELECT
    m.name AS table_name, --the name of each table in the database.
    GROUP_CONCAT(p.name, CHAR(10)) AS columns_list
    --joins all column names from that table into one text block, separated by line breaks (CHAR(10) = newline)
FROM sqlite_master AS m --system table built into SQLite.It contains metadata about everything in the database
JOIN pragma_table_info(m.name) AS p --special SQLite command that returns information about the columns of a given table
WHERE m.type = 'table'
GROUP BY m.name
ORDER BY m.name;
"""
df = pd.read_sql_query(query, conn)
for _, row in df.iterrows():
    print(f"\n {row['table_name']}")
    print(row['columns_list'])



 Ball_by_Ball
Match_Id
Over_Id
Ball_Id
Innings_No
Team_Batting
Team_Bowling
Striker_Batting_Position
Striker
Non_Striker
Bowler

 Batsman_Scored
Match_Id
Over_Id
Ball_Id
Runs_Scored
Innings_No

 Batting_Style
Batting_Id
Batting_hand

 Bowling_Style
Bowling_Id
Bowling_skill

 City
City_Id
City_Name
Country_id

 Country
Country_Id
Country_Name

 Extra_Runs
Match_Id
Over_Id
Ball_Id
Extra_Type_Id
Extra_Runs
Innings_No

 Extra_Type
Extra_Id
Extra_Name

 Match
Match_Id
Team_1
Team_2
Match_Date
Season_Id
Venue_Id
Toss_Winner
Toss_Decide
Win_Type
Win_Margin
Outcome_type
Match_Winner
Man_of_the_Match

 Out_Type
Out_Id
Out_Name

 Outcome
Outcome_Id
Outcome_Type

 Player
Player_Id
Player_Name
DOB
Batting_hand
Bowling_skill
Country_Name

 Player_Match
Match_Id
Player_Id
Role_Id
Team_Id

 Rolee
Role_Id
Role_Desc

 Season
Season_Id
Man_of_the_Series
Orange_Cap
Purple_Cap
Season_Year

 Team
Team_Id
Team_Name

 Toss_Decision
Toss_Id
Toss_Name

 Umpire
Umpire_Id
Umpire_Name
Umpire_Country

 Venue
Venu

In [ ]:
def run_sql(sql, head=10):
    """Executes SQL and outputs the result as a table."""
    with engine.connect() as conn:
        res = pd.read_sql_query(sql, conn) #returns the result as a DataFrame (pandas table).
    print(" Query completed:")
    print(sql)
    print("\n Result:")
    display(res.head(head)) #The display() function is used in Google Colab/Jupyter to beautifully display the DataFrame as a table.
    return res #Returns the entire DataFrame res so that you can continue to use the result in Python (for example, save it to CSV, filter, analyze, etc.).


In [ ]:
# Query 1 — Select All Columns from Player_Match
q1 = "SELECT * FROM Player_Match;"
res1 = run_sql(q1)



In [ ]:
# Query 2 — Total Runs by Each Batsman
q2 = """
SELECT
    P.Player_Name AS batsman_Name,
    SUM(bs.Runs_Scored) AS total_runs
FROM Ball_by_Ball AS bb
JOIN Batsman_Scored AS bs
    ON bb.Match_Id = bs.Match_Id
   AND bb.Over_Id = bs.Over_Id
   AND bb.Ball_Id = bs.Ball_Id
   AND bb.Innings_No = bs.Innings_No
JOIN Player AS p
    ON bb.Striker = p.Player_Id
GROUP BY bb.Striker
ORDER BY total_runs DESC;

"""
res2 = run_sql(q2)

In [ ]:
# Query 3 — Fifties and Hundreds per Batsman
q3 = """
WITH player_match_runs AS (
    SELECT
        p.Player_Name AS batsman_name,
        bb.Match_Id,
        SUM(bs.Runs_Scored) AS runs_in_match
    FROM Ball_by_Ball AS bb
    JOIN Batsman_Scored AS bs
        ON bb.Match_Id = bs.Match_Id
       AND bb.Over_Id = bs.Over_Id
       AND bb.Ball_Id = bs.Ball_Id
       AND bb.Innings_No = bs.Innings_No
    JOIN Player AS p
        ON bb.Striker = p.Player_Id
    GROUP BY bb.Striker, bb.Match_Id
)
SELECT
    batsman_name,
    SUM(CASE WHEN runs_in_match BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS fifties,
    SUM(CASE WHEN runs_in_match >= 100 THEN 1 ELSE 0 END) AS hundreds
FROM player_match_runs
GROUP BY batsman_name
ORDER BY hundreds DESC, fifties DESC;
"""
res3 = run_sql(q3)


In [ ]:
# Query 4 — Best Bowling Figures
q4 = """
WITH bowling_stats AS (
    SELECT
        p.Player_Name AS bowler_name,
        bb.Match_Id,
        SUM(bs.Runs_Scored) AS runs_conceded,
        COUNT(wt.Player_Out) AS wickets
    FROM Ball_by_Ball AS bb
    JOIN Player p
        ON bb.Bowler = p.Player_Id
    LEFT JOIN Batsman_Scored AS bs
        ON bb.Match_Id = bs.Match_Id
       AND bb.Over_Id = bs.Over_Id
       AND bb.Ball_Id = bs.Ball_Id
       AND bb.Innings_No = bs.Innings_No
    LEFT JOIN Wicket_Taken AS wt
        ON bb.Match_Id = wt.Match_Id
       AND bb.Over_Id = wt.Over_Id
       AND bb.Ball_Id = wt.Ball_Id
       AND bb.Innings_No = wt.Innings_No
    GROUP BY bb.Bowler, bb.Match_Id
)
SELECT
    bowler_name,
    MAX(wickets) AS best_wickets,
    MIN(runs_conceded) AS least_runs_in_best
FROM bowling_stats
GROUP BY bowler_name
ORDER BY best_wickets DESC;

"""
res4 = run_sql(q4)